> (Última Actualización: 18 de Noviembre de 2025)

# **Introducción al modelado continuo** (*a.k.a.* **Ecuaciones de la física matemática**)
## Laboratorio numérico

Bienvenidos al laboratorio numérico de la materia *Introducción al modelado continuo*, también conocida como *Ecuaciones de la física matemática*. En este, vamos a ver métodos numéricos para resolver distintos tipos de problemas de manera general, y su relación y aplicación a problemas específicos de la materia, de manera tal de complementar los contenidos de la cursada teórico-práctica.

# Motivación de este colab

En el 1er eje temático de la materia vimos ecuaciones diferenciales ordinarias (ODEs), que describían la dinámica de variables continuas que dependían sólo del tiempo, es decir, cada una de las variables podía representarse con una función univariada (depende de una única variable). Estos sistemas los podíamos resolver integrando numéricamente de manera tal de obtener la solución como la evolución temporal de las variables, en el marco del problema de valores iniciales.

En este eje temático abordamos un problema distinto, que consiste en tener variables continuas que se representan como funciones multivariadas $u(x,t)$. De esta manera, la derivada respecto de una de sus variables ya no es la derivada total, si no que es una derivada parcial. Ahora las reglas que describen la evolución temporal de estas variables ya no son ODEs, si no que son **ecuaciones en derivadas parciales** (PDEs), que son básicamente ecuaciones que relacionan funciones multivariadas y sus derivadas parciales. La resolución de este tipo de sistemas se encuadra en el marco de problemas de condiciones de borde, siendo un caso particular la consideración de condiciones iniciales cuando una de las variables es el tiempo.

Las PDEs son fundamentales en muchas áreas de la física y las matemáticas aplicadas, describiendo muchísimos fenómenos diversos. En este colab vamos a ver algunas PDEs clásicas dentro de lo que son la **Ecuaciones de la Física Matemática**, como las que describen la difusión del calor, las ondas, y procesos estacionarios. La resolución de estos sistemas se vuelve altamente no trivial, debiendo recurrir a distintas estrategias matemáticas (como las vistas en la teórica) y numéricas.

En este Colab vamos a revisitar los métodos numéricos vistos en las últimas clases (**diferencias finitas y métodos espectrales**) para la resolución de problemas definidos a partir de estas ecuaciones paradigmáticas bajo ciertas condiciones de contorno.

In [ ]:
import numpy as np
np.set_printoptions(legacy='1.25')
import matplotlib.pyplot as plt
import scipy as sp

# Retina for matplotlib
%config InlineBackend.figure_format = 'retina'
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "cm"
plt.rcParams["figure.dpi"] = 120
plt.rcParams["legend.fontsize"] = "medium"
plt.rcParams["axes.labelsize"] = "large"

---
# Ecuaciones de la Física Matemática

Las ecuaciones canónicas de la física matemática son un conjunto de ecuaciones que representan fenómenos fundamentales y aparecen en muchas áreas de la ciencia. Estas ecuaciones surgen en gran parte porque describen los comportamientos más básicos y universales de los sistemas físicos. Se trata generalmente de ecuaciones diferenciales en derivadas parciales (PDEs) que incluyen derivadas primeras y segundas en espacio y/o tiempo. Las derivadas segundas son frecuentes porque describen curvatura, aceleración, difusión o equilibrio, y permiten modelar vibraciones, ondas, temperatura, potenciales, fluidos, etc. Muchas leyes fundamentales (como la conservación de energía, momentum, o carga) conducen a estas formas matemáticas. Además, estas PDEs suelen ser lineales, lo que facilita su análisis y solución en comparación con ecuaciones no lineales más complejas. Las principales son:

- **Ecuación de difusión/calor:**
$$
u_t = \alpha u_{xx}
$$
- **Ecuación de ondas:**
$$
u_{tt} = c^2 u_{xx}
$$
- **Ecuación de Laplace:**
$$
\nabla^2 u = 0
$$
- **Ecuación de Poisson:**
$$
\nabla^2 u = f(x, y)
$$

Las ecuaciones se clasifican en elípticas, parabólicas e hiperbólicas. Esta clasificación viene de estudiar cómo se comportan las soluciones cerca de un punto y de la forma del desarrollo cuadrático que aparece al linealizar la ecuación (el mismo tipo de clasificación que se usa en cónicas: elipse, parábola, hipérbola).

Para una PDE lineal de segundo orden en dos variables,
$$
A u_{xx} + B u_{xy} + C u_{yy} + \dots = 0,
$$
el tipo depende del discriminante $B^{2}-4AC$, igual que en las cónicas:

$B^{2}-4AC < 0$ → elíptica (como una elipse).

$B^{2}-4AC = 0$ → parabólica (como una parábola).

$B^{2}-4AC > 0$ → hiperbólica (como una hipérbola).

La forma del discriminante describe cómo se comportan las “direcciones principales” de la ecuación (cómo se comunican los puntos con sus vecinos). Esa geometría es lo que determina si la ecuación difunde, se propaga o está en equilibrio. Aunque el origen matemático es formal, la interpretación física es muy simple y extremadamente útil para entender qué condiciones de contorno necesita cada problema y qué comportamiento esperar de la solución.

1. **Ecuaciones Elípticas**:

$$
\nabla^2 u = 0 \quad \text{(Laplace)} \\
\nabla^2 u = f \quad \text{(Poisson)}
$$
Las ecuaciones son elípticas, es decir, que todas las direcciones están mezcladas. La forma cuadrática es definida positiva o negativa. En términos "geométricos" esto significa que no hay direcciones preferenciales, si no que cada punto “mira” a todos sus vecinos por igual, y la información se difunde instantáneamente por todo el dominio. Esto explica por qué Laplace necesita condiciones en todo el borde, y por qué sus soluciones son suaves.

2. **Ecuaciones Parabólicas**:

$$
u_t = D \nabla^2 u \quad \text{(calor / difusión)}
$$

Una ecuación es parabólica cuando, desde el punto de vista matemático, el operador de segundo orden tiene un determinante nulo, lo que implica la existencia de una única dirección característica real. Esa dirección funciona como una "línea privilegiada" a lo largo de la cual la ecuación puede reinterpretarse localmente como un proceso de difusión. En estas ecuaciones, cualquier estado inicial tiende a suavizarse con el tiempo. No aparecen frentes abruptos ni propagación con velocidad definida, sino un alisamiento progresivo del campo. Por eso, las ecuaciones parabólicas se condicionan mediante una condición inicial, que fija el estado de partida, junto con condiciones de contorno, que determinan cómo se distribuye la difusión dentro del dominio.


3. **Ecuaciones Hiperbólicas**:

$$
u_{tt} = c^2 u_{xx} \quad \text{(ondas)}
$$

La ecuación es hiperbólica, lo que implica la existencia de direcciones privilegiadas de propagación. Matemáticamente, el operador de derivadas de segundo orden puede factorizarse en dos derivadas direccionales reales y distintas, lo que define dos familias de características. Como consecuencia, la información se propaga únicamente a lo largo de esas trayectorias, generando ondas y frentes de onda que avanzan de manera ordenada y con una velocidad finita bien definida. Por esta razón, las ecuaciones hiperbólicas requieren condiciones iniciales (y no sólo de contorno) para determinar una solución física, porque su dinámica está gobernada por la propagación temporal a lo largo de características.

**En resumen:**

| Tipo  | Ejemplo típico    | Física que describe  | Orden en t | Orden en x | Condiciones requeridas| Comportamiento
| ----- | ------- | ------ | ------- | ----- | ---- | ---- |
| **Elíptica**    | $( \nabla^2 u = f )$     | Equilibrio estático | — | 2 | Solo condiciones de contorno (en todo el borde) | No hay propagación; soluciones suaves |
| **Parabólica**  | $( u_t = D\nabla^2 u )$  | Difusión / relajación | 1 | 2 | 1 condición inicial (en t) + 2 condiciones de contorno (en x) | Suavizan perturbaciones |
| **Hiperbólica** | $( u_{tt} = c^2u_{xx} )$ | Ondas | 2 | 2 | 2 condiciones iniciales (u y ∂u/∂t) + 2 condiciones de contorno | Propagación con velocidad finita |

---
# Métodos numéricos para PDEs

Existen varias técnicas que se aplican ampliamente en el campo de las PDEs. Los métodos más utilizados incluyen:

- **Métodos de diferencias finitas**: Calculan derivadas mediante diferencias entre valores de la función en puntos de una grilla. Son comunes para ecuaciones de difusión, onda y Poisson.

- **Método de volúmenes finitos**: Calcula las soluciones en volúmenes discretos (o celdas) en lugar de puntos específicos de una grilla. Se usa mucho en dinámica de fluidos y problemas de conservación, ya que garantiza la conservación de cantidades físicas (como masa o energía) en cada celda.

- **Método de elementos finitos**: Divide el dominio en elementos pequeños y usa funciones de prueba para aproximar la solución dentro de cada elemento. Es muy flexible y se usa para resolver PDEs en dominios complejos o con condiciones de contorno complicadas.

- **Métodos espectrales**: Aproximan la solución como una serie de funciones base, como polinomios o senos y cosenos, usando la Transformada de Fourier o de Chebyshev. Son útiles para problemas con geometrías simples y donde se requiere alta precisión.

Varios de estos métodos pueden implementarse con **esquemas explícitos e implícitos**, haciendo referencia a cómo se avanza en el tiempo al resolver la ecuación, dependiendo el esquema de cómo aproximamos las derivadas, cómo armamos la grilla, y de cómo organizamos los términos en la ecuación.

En este Colab vamos a ver aplicaciones de **métodos de diferencias finitas** y de **métodos espectrales**, los cuales ya presentamos y discutimos en clases pasadas, en el marco de la materia.

---
## Método de diferencias finitas

Existen varios esquemas con los que se puede implementar un método de diferencias finitas a una ecuación diferencial en derivadas parciales que involucre el tiempo y el espacio como variables, tal como lo hacen las ecuaciones típicas asociadas a sistemas físicos.

La decisión sobre como aproximar las derivadas temporales, hacia adelante (forward) o hacia atrás (backward), termina siendo determinante de que el método de avance temporal sea [explícito o implícito](https://en.wikipedia.org/wiki/Explicit_and_implicit_methods).

| Nombre del esquema | Tiempo | Espacio | Tipo general |
|---------------------|---------|----------|------------|
| FTCS (Forward Time, Centered Space) | Forward | Centered | Explícito | Simple, pero inestable para muchas ecuaciones (por ejemplo, la del calor). |
| FTFS (Forward Time, Forward Space) | Forward | Forward | Explícito |
| FTBS (Forward Time, Backward Space) | Forward | Backward | Explícito |
| BTCS (Backward Time, Centered Space) | Backward | Centered | Implícito |
| BTFS / BTBS | Backward | Forward / Backward | Implícito |
| Crank–Nicolson | Promedio de FTCS y BTCS | Centered | Semi-implícito |

Los distintos esquemas tienen ventajas y desventajas, y son aplicables según el tipo de PDE que querramos resolver.

En la **ecuación de difusión** los modos de alta frecuencia se suavizan, lo que vuelve a los esquemas explícitos muy sensibles a la estabilidad. Por eso aparece la condición CFL difusiva, que en diferencias finitas toma la forma clásica ( $\alpha \Delta t / \Delta x^{2} \le C $), garantizando que el suavizado numérico no sea más débil que el físico. Cuando no se cumple, el esquema explícito puede volverse inestable incluso para mallas finas. Por este motivo se prefieren métodos implícitos como Euler implícito o Crank-Nicolson, que permiten avanzar con pasos más grandes sin perder estabilidad, aunque a costa de resolver sistemas lineales.

La **ecuación de ondas** transporta energía sin disipación, y los esquemas centrados en tiempo y espacio (como leapfrog o Lax-Wendroff) se usan porque respetan mejor esa estructura propagativa. Aquí la condición CFL expresa una limitación física y numérica: ($c \Delta t / \Delta x \le C$), asegurando que la discretización no permita que la solución “avance” más rápido que la velocidad real de la onda. Si la CFL se viola, el esquema explícito desarrolla inestabilidad o dispersión excesiva. Este vínculo entre propagación física y CFL es característico de métodos explícitos de diferencias finitas.

En las **ecuaciones de Laplace y Poisson** no hay evolución temporal, así que la CFL no interviene. Al discretizar se obtiene un sistema lineal esparso con la estructura típica del operador Laplaciano. Para resolverlo se usan métodos iterativos estacionarios como Jacobi, Gauss-Seidel o SOR, que aprovechan la naturaleza definida positiva del problema y son fáciles de implementar, aunque a veces convergen lentamente. También existen métodos directos, como LU, Cholesky o factorizaciones especializadas para matrices banda, que son muy eficientes en 1D o para dominios pequeños porque dan la solución del sistema en un solo paso; sin embargo, en 2D y 3D su costo en memoria y tiempo crece rápidamente, por lo que los iterativos suelen ser la opción preferida.

Los esquemas típicos según las ecuaciones diferenciales son:

| Tipo de ecuación | Ecuación modelo | Esquemas típicos | Naturaleza | Comentario |
|------------------|-----------------|------------------|------------|------------|
| **Difusión (parabólica)** | ∂u/∂t = α ∂²u/∂x² | FTCS | Explícito | Simple pero condicionalmente estable (CFL: Δt ≤ Δx²/(2α)). |
| | | BTCS | Implícito | Incondicionalmente estable; requiere resolver sistema tridiagonal. |
| | | Crank-Nicolson | Semi-implícito | Segundo orden en tiempo y espacio; muy usado. |
|------------------|-----------------|------------------|------------|------------|
| **Ondas (hiperbólica)** | ∂²u/∂t² = c² ∂²u/∂x² | Centradas (leapfrog) | Explícito | Naturales por simetría; requieren CFL: cΔt ≤ Δx. |
| | | Lax-Wendroff | Explícito | Más preciso; puede tener oscilaciones numéricas. |
|------------------|-----------------|------------------|------------|------------|
| **Laplace / Poisson (elípticas)** | ∂²u/∂x² + ∂²u/∂y² = f | Iterativos: Jacobi, Gauss-Seidel, SOR | Iterativos | SOR acelera la convergencia; Jacobi es lento. |
| | | Directos (LU/Cholesky tridiagonal) | Directos | Factible solo en mallas chicas/estructuradas. |

---
## Métodos espectrales

Las tres ecuaciones (difusión, ondas y Laplace/Poisson) pueden resolverse también con métodos espectrales, aprovechando que la discretización en espacio convierte derivadas en multiplicaciones por números (o matrices muy simples) en el dominio de Fourier o Chebyshev. Para la difusión y las ondas, esto produce esquemas temporales extraordinariamente precisos y con errores dispersivos y disipativos controlados, siempre que las condiciones de contorno sean compatibles (periódicas para Fourier, o bien dominios pequeños y suaves para Chebyshev). Para Laplace y Poisson, los métodos espectrales permiten resolver el sistema espacial prácticamente de manera directa usando la estructura diagonal (Fourier) o casi diagonal (Chebyshev) del operador, logrando convergencia exponencial. Los esquemas típicos son pseudospectrales, combinando transformadas FFT con avances temporales (cuando aplican) o con resolución directa del sistema elíptico. Su principal limitación es que requieren geometrías y contornos muy regulares; fuera de eso, pierden la ventaja estructural.

Los esquemas típicos espectrales según el tipo de PDE son:

| Tipo de ecuación | Representación típica | Esquemas temporales | Comentario |
|------------------|------------------------|----------------------|------------|
| **Difusión (parabólica)** | Fourier o senos/cosenos: u(x,t) = Σ û_k(t) φ_k(x) | Euler implícito por modo | Estable; cada modo satisface d û_k/dt = -α k² û_k. |
| | | Crank-Nicolson espectral | Segundo orden; muy usado. |
| | | Integración exacta por modo | û_k(t+Δt) = exp(-α k² Δt) û_k(t); muy eficiente. |
|------------------|------------------------|----------------------|------------|
| **Ondas (hiperbólica)** | Fourier o senos/cosenos | Leapfrog espectral | Version centrada por modo; requiere CFL si es explícito. |
| | | Newmark / Verlet espectral | Estables y precisos en el espacio espectral. |
| | | Integración exacta por modo | Solución exacta del oscilador: û_k(t)=A cos(ckt)+B sin(ckt). |
|------------------|------------------------|----------------------|------------|
| **Laplace / Poisson (elípticas)** | Fourier o senos/cosenos | Resolución directa por modo | -(k_x²+k_y²) û = f̂ → û = f̂ / (k²). Muy rápido. |
| | | Transformadas mixta (ej: Fourier + discretización finita) | — | Útil si solo algunas direcciones son periódicas. |


---
# Ecuación de Difusión/Calor

Consideremos la ecuación de difusión unidimensional:
$$
u_t = \alpha u_{xx}
$$

Describe procesos que suavizan y disipan, como la difusión de temperatura o partículas. La información fluye hacia adelante en el tiempo, no hacia atrás. Requiere una condición inicial y condiciones de contorno en todo momento. Un ejemplo podría ser tirar tinta en agua, que evoluciona en tiempo y se difunde, se homogeniza.


---
## FTCS

El esquema [Forward Time Centred Space](https://en.wikipedia.org/wiki/FTCS_scheme) (**FTCS**) es una combinación específica de diferencias finitas para resolver ecuaciones en derivadas parciales (PDEs), que se caracteriza por usar diferencias progresivas (forward) en el tiempo y diferencias centradas en el espacio.

La ecuación se puede discretizar en una malla de puntos en el espacio $x$ y en el tiempo $t$. Utilizando una malla con espaciamiento $\Delta x$ y $\Delta t$, las aproximaciones de diferencias finitas nos llevan a la siguiente formulación:

$$
\frac{u_i^{n+1} - u_i^n}{\Delta t} = \alpha \frac{u_{i+1}^n - 2u_i^n + u_{i-1}^n}{(\Delta x)^2}
$$

Despejando $u_i^{n+1}$:
$$
u_i^{n+1} = u_i^n + \frac{\alpha \Delta t}{(\Delta x)^2} \left( u_{i+1}^n - 2u_i^n + u_{i-1}^n \right)
$$

Esta fórmula permite calcular el valor de $u$ en cada punto de la grilla en el próximo tiempo $n + 1$, utilizando los valores en el tiempo actual $n$. Por este motivo, se trata de un método explícito. Podemos avanzar en el tiempo sin necesidad de resolver un sistema de ecuaciones en cada paso, lo cual hace que el método sea relativamente simple y rápido de implementar.

Una limitación importante del método FTCS es su estabilidad. Este método es condicionalmente estable para la ecuación de difusión: es estable sólo si se cumple una condición sobre los pasos de tiempo y espacio, conocida como la **condición de Courant-Friedrichs-Lewy (CFL)**:
$$
\alpha \frac{\Delta t}{(\Delta x)^2} \leq \frac{1}{2}
$$
Si esta condición no se cumple, el método puede volverse inestable, causando que la solución diverja con oscilaciones crecientes. Esta limitación hace que FTCS sea más adecuado para situaciones donde $\Delta t$ puede ser suficientemente pequeño.


**Ejemplo**

Resolvamos la ecuación de difusión en un dominio unidimensional con una condición inicial de pulso central. La solución numérica se obtiene utilizando el método de diferencias finitas FTCS

In [ ]:
# Parámetros de la malla
L = 1.0  # Longitud del dominio
T = 0.3  # Tiempo total de simulación
Nx = 128  # Número de puntos en el espacio
Nt = 300  # Número de puntos en el tiempo
alpha = 0.01  # Coeficiente de difusión
dx = L / (Nx - 1)
dt = T / Nt
r = alpha * dt / dx**2
# Inicialización de la malla
x = np.linspace(0, L, Nx)
u = np.zeros((Nx, Nt), dtype=float)
# Condiciones iniciales
u[int(Nx/4):int(3*Nx/4), 0] = 1.0  # Pulso inicial en el centro
# Iteración temporal
for n in range(Nt - 1):
    for i in range(1, Nx - 1):
        u[i, n+1] = u[i, n] + r * (u[i+1, n] - 2*u[i, n] + u[i-1, n])
    # Este doble for implementa el paso descritp arriba...
    # Se les ocurre una forma más eficiente de realizar las mismas operaciones?


# Visualización del resultado
fig, axs = plt.subplots(2, 1, figsize=(10, 6), constrained_layout=True)
axs[0].plot(x, u[:, 0], ".-", mec="k", mew=0.5, label="$u(x,0)$", ds="steps-mid")
axs[0].grid()
axs[0].legend()
axs[0].set_title("Condición inicial")
axs[0].set_xlabel("$x$")
for n in range(1, Nt + 1, 50):
    axs[1].plot(x, u[:, n+1], ".-", c=f"C{n//50}", mec=f"C{n//50}", mew=0.5, label=f"$u(x,{n*dt})$")
axs[1].grid()
# axs[1].legend()
axs[1].set_title("Evolución en el tiempo")
axs[1].set_xlabel("$x$")

plt.show()

In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML

# Parámetros de la malla
L = 1.0  # Longitud del dominio
T = 0.3  # Tiempo total de simulación
Nx = 128  # Número de puntos en el espacio
Nt = 300  # Número de puntos en el tiempo
alpha = 0.01  # Coeficiente de difusión
dx = L / (Nx - 1)
dt = T / Nt
r = alpha * dt / dx**2

# Inicialización de la malla
x = np.linspace(0, L, Nx)
u = np.zeros((Nx, Nt), dtype=float)

# Condiciones iniciales
u[int(Nx/4):int(3*Nx/4), 0] = 1.0  # Pulso inicial en el centro

# Iteración temporal
for n in range(Nt - 1):
    for i in range(1, Nx - 1):
        u[i, n+1] = u[i, n] + r * (u[i+1, n] - 2*u[i, n] + u[i-1, n])

# Inicializa el plot
fig, ax = plt.subplots(figsize=(10, 6))
line, = ax.plot([], [], lw=2)
ax.set_xlim(0, L)
ax.set_ylim(0, 1.2)
ax.set_xlabel('Posición x')
ax.set_ylabel('u(x, t)')
ax.set_title('Solución de la ecuación de difusión')

# Función de animación
def animate(frame):
    line.set_data(x, u[:, frame])
    ax.set_title(f'Solución de la ecuación de difusión - Tiempo: {frame*dt:.3f}')
    return line,

# Crear animación
print("Creando animación...")
ani = animation.FuncAnimation(fig, animate, frames=Nt, interval=24/Nt, blit=True)

print("Renderizando video:")
# Para Google Colab - mostrar como HTML
display(HTML(ani.to_jshtml()))

---
### Ejercicio 1

Se desea resolver la ecuación del calor con condiciones de Dirichlet homogéneas:

$$
\left\{\begin{array}{rcll}
u_t(x,t)& = & u_{xx}(x,t)& \textrm{ para } x\in[-1,1],\; t\in[0,T]\\
u(-1,t) &= &u(1,t)=0\, &\forall t
\end{array}\right.
$$

Para ello se propone un esquema centrado en el espacio y explícito en el tiempo:

$$
\frac{U_{j}^{n+1}-U_j^n}{\Delta t} =  \frac{U_{j+1}^n-2U_j^n+U_{j-1}^n}{\Delta x^2}.
$$

Considerar la siguiente condición inicial:
$$
u_0(x) = \left\{\begin{array}{rl}
					x+1 & \textrm{ si }x<0 \\
					1-x  & \textrm{ si }x\ge 0
					\end{array}\right.,
					\quad\quad
$$

1.1. Escribir el esquema de forma matricial: $U^{n+1} = AU^n$ para una matriz $A$ adecuada, tomando un $T=1$, y resolver numéricamente para $\Delta x=0.05$, y $\Delta t=0.0012$. Graficar.

1.2. Resolver el mismo problema que antes, pero ahora para $\Delta t=0.0013$. Qué observa?


In [ ]:
# # # COMPLETAR

---
### Ejercicio 2

Estudiemos la transferencia de calor, definida por la ecuación

$$ \frac{\partial T}{\partial t}=k\frac{\partial^2 T}{\partial x^2} $$

Discretizamos el sistemas y aplicamos el método FTCS:

$$\frac{T_{i}^{n+1}-T_{i}^{n}}{\Delta t}=k\frac{T_{i+1}^{n}-2T_{i}^{n}+T_{i-1}^{n}}{\Delta x^2},\\\\\\
T_{i}^{n+1}=T_{i}^{n}+\frac{k\Delta t}{\Delta x^2}\Big(T_{i+1}^{n}-2T_{i}^{n}+T_{i-1}^{n}\Big).$$

A partir de la condición CFL, sabemos que para que la solución sea estable, debemos asegurarnos de que:

$$\Delta t \leq \frac{\Delta x^2}{2k}.$$

2.1. Si tengo una barra de longitud $1 m$, que voy a discretizar en 100 intervalos (101 puntos). Qué consideraciones sobre el paso temporal tengo que tener? Puedo elegir cualquier valor?

2.2. Considere que la temperatura inicial de la barra se fija en $100 °C$, pero en los extremos se mantendrá constante en $0 °C$; y que estpa hecha de un material que tiene una conductividad térmica con un valor de $k=0.01$. Estudie la evolución de la temperatura, para un paso temporal que considere, hasta el momento en el que la temperatura en la mitad de la barra es de $50 °C$.

In [ ]:
# # # COMPLETAR

---
## BTCS

El esquema **BTCS (Backward-Time Central-Space)** es uno de los métodos más utilizados para resolver la ecuación de difusión (o calor) con diferencias finitas, porque lidia con el principal problema del esquema FTCS, que es la inestabilidad. Es un método implícito, lo que significa que, para encontrar la solución en el nuevo tiempo ($t^{n+1}$), se debe resolver un sistema de ecuaciones lineales simultáneamente en lugar de calcular cada punto de manera independiente.

El esquema BTCS se basa en aproximar la derivada temporal con el método de diferencias finitas regresivo (Backward Time), evaluada en el nodo $i$:
$$\frac{\partial u}{\partial t} \approx \frac{u_i^{n+1} - u_i^n}{\Delta t}$$

Para la derivada espacial ($\partial^2 u / \partial x^2$) se usa una diferencia central (Central Space), pero evaluada en el nuevo nivel de tiempo $n+1$:
$$\frac{\partial^2 u}{\partial x^2} \approx \frac{u_{i+1}^{n+1} - 2u_i^{n+1} + u_{i-1}^{n+1}}{\Delta x^2}$$

Al sustituir y reagrupar los términos, el esquema queda:

$$-r u_{i-1}^{n+1} + (1 + 2r) u_i^{n+1} - r u_{i+1}^{n+1} = u_i^n$$

Donde $r = \frac{\alpha \Delta t}{\Delta x^2}$ es el número de difusión.

La gran ventaja del BTCS es que es incondicionalmente estable. Esto significa que se puede elegir un paso de tiempo $\Delta t$ tan grande como se quiera (limitado sólo por la precisión, no por la estabilidad) sin que la solución numérica explote, en contraste con el esquema FTCS.

Para cada paso de tiempo, la ecuación de BTCS debe aplicarse a todos los puntos interiores de la grilla, lo que genera un sistema de ecuaciones lineales de la forma:

$$ A u^{n+1} = b$$

  * $u^{n+1}$ es el vector de valores de temperatura desconocidos en el tiempo $n+1$.
  * $b$ es el vector de valores de temperatura conocidos en el tiempo anterior $n$ (vector **$\mathbf{u}^n$**, ajustado por las condiciones de contorno si no son cero).
  * $A$ es la matriz de coeficientes, que es una matriz tridiagonal y no cambia con el tiempo (si $r$ es constante).

La matriz $A$ para los $N$ puntos interiores es:

$$A = \begin{pmatrix}
1+2r & -r & 0 & \dots & 0 \\
-r & 1+2r & -r & \dots & 0 \\
0 & -r & 1+2r & \dots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & -r & 1+2r
\end{pmatrix}$$

El corazón del algoritmo es resolver este sistema en cada paso temporal.

Resolvamos para condiciones de contorno de Dirichlet homogéneas ($u=0$ en ambos extremos), . Para esto, vamos a resolver el sistema de ecuaciones usando `numpy.linalg.solve`, aprovechando la estructura tridiagonal de la matriz $A$.



In [ ]:
alpha = 0.5    # Coeficiente de difusividad (ej. m^2/s)
L = 1.0        # Longitud del dominio (m)
T_max = 0.1    # Tiempo total de simulación (s)
Nx = 100        # Número de divisiones en el espacio
dx = L / Nx    # Tamaño de paso en el espacio
Nt = 1000      # Número de pasos en el tiempo
dt = T_max / Nt # Tamaño de paso en el tiempo
N_interior = Nx - 1

# Parámetro de difusión r
r = alpha * dt / (dx**2)
print(f"Parámetro de difusión r = {r:.4f}")
print(f"Estabilidad: BTCS es estable para cualquier r.")

# Puntos de la grilla (incluyendo las fronteras x=0 y x=L)
x = np.linspace(0, L, Nx + 1)

# Vector de solución en el tiempo n (u_curr)
u_curr = np.zeros(Nx + 1)
u_next = np.zeros(Nx + 1)

# Condición inicial (Ej. una perturbación gaussiana)
mu = 0.5  # Centro
sigma = 0.05 # Ancho
u_curr[:] = np.exp(-((x - mu) / sigma)**2)

# Aplicar las condiciones de contorno iniciales (Dirichlet u=0)
u_curr[0] = 0.0
u_curr[Nx] = 0.0

# Matriz
A = np.zeros((N_interior, N_interior))
diag = 1 + 2 * r  # Elementos en la diagonal principal
off_diag = -r     # Elementos en las diagonales secundarias
for i in range(N_interior):
    A[i, i] = diag
    if i > 0:
        A[i, i-1] = off_diag # Sub-diagonal
    if i < N_interior - 1:
        A[i, i+1] = off_diag # Super-diagonal

# Resuelvo
u_sol_history = [u_curr.copy()]
for n in range(Nt):
    # Vector del lado derecho (b) del sistema A * u^(n+1) = b.
    # Para Dirichlet homogéneo, b es simplemente el vector interior de u^n.
    b = u_curr[1:Nx].copy()

    # Si las condiciones de contorno no fueran cero, se modificaría b:
    # b[0] += r * u_curr[0]
    # b[N_interior - 1] += r * u_curr[Nx]

    # Resolver el sistema lineal A * u_interior^(n+1) = b
    # np.linalg.solve es eficiente para matrices pequeñas y tridiagonales
    u_next_interior = np.linalg.solve(A, b)
    # Asignar los valores resueltos a los puntos interiores
    u_next[1:Nx] = u_next_interior
    # Aplicar las condiciones de contorno (Dirichlet homogéneo)
    u_next[0] = 0.0
    u_next[Nx] = 0.0
    # Avanzar el tiempo
    u_curr[:] = u_next[:]

    # Guardar algunas soluciones para graficar
    if n % (Nt // 10) == 0 or n == Nt - 1:
         u_sol_history.append(u_curr.copy())

plt.figure(figsize=(6, 4))
plt.title(f"Solución de la Ecuación de Difusión (Esquema BTCS)")
plt.xlabel("Posición $x$")
plt.ylabel("Concentración $u(x, t)$")
time_points = np.linspace(0, T_max, len(u_sol_history))
for i, u_snap in enumerate(u_sol_history):
    plt.plot(x, u_snap, label=f't = {time_points[i]:.3f} s', alpha=0.8)
plt.legend(loc='upper right')
plt.grid(True)
plt.show()

---
## Métodos espectrales para la ecuación de calor unidimensional

La base de la transformada de Fourier es ideal para resolver la ecuación del calor. En una dimensión espacial, la ecuación del calor viene dada por:
$$ u_{t} = α^{2}u_{xx}$$
donde $u(t,x)$ es la distribución de temperatura en el tiempo y el espacio. Si aplicamos la transformada de Fourier en el espacio, entonces $F(u(t,x)) = \hat{u}(t,\omega)$. La ecuación diferencial parcial se convierte en:
$$ \hat{u}_{t} = -\alpha^{2}\omega^{2}\hat{u}$$
Dado que las dos derivadas espaciales contribuyen en $(i\omega)^{2} = -\omega^{2}$ en el dominio de la transformada de Fourier, al aplicar la transformada de Fourier, la PDE se convierte en una ODE para cada frecuencia fija $\omega$.

Vamos a resolver numéricamente usando un método espectral explícito. Para esto transformamos la ecuación al dominio de frecuencias mediante la FFT, resultando en la ecuación $ \hat{u}_{t} = -\alpha^{2}\kappa^{2}\hat{u}$, donde $\kappa$ es la frecuencia discretizada. Es importante usar fftshift para reordenar los números de onda según la convención. Dado que la PDE es lineal, es posible avanzar explícitamente en el tiempo el sistema utilizando un integrador, por ejemplo con `solve_ivp`.

En las figuras se muestran diferentes vistas de la distribución de temperatura $u(t,x)$ a medida que evoluciona en el tiempo. Es evidente que las frecuencias más altas, correspondientes a valores mayores de $\omega$, decaen más rápidamente con el tiempo, de modo que los picos pronunciados en la distribución de temperatura se suavizan rápidamente. Esto se observa claramente en las figuras, siendo que las esquinas pronunciadas se difuminan rápidamente, ya que corresponden a los números de onda más altos. Eventualmente, las variaciones de los números de onda más bajos también decaen, hasta que la temperatura alcanza una distribución constante en estado estacionario, que es una solución de la ecuación de Laplace $u_{xx} = 0$. Al resolver esta PDE mediante la FFT, asumimos que el dominio de la solución es periódico, de modo que se identifican los límites derecho e izquierdo y el dominio forma un anillo. Sin embargo, si el dominio es suficientemente grande, el efecto de los bordes es pequeño. No obstante, si el dominio es pequeño, esta periodicidad puede introducir artefactos no físicos, como por ejemplo calor que "sale" por la derecha y "entra" por la izquierda. Pero si el dominio es grande y la solución está concentrada en una región central, entonces los extremos están lejos y no afectan significativamente la evolución, y es como si los bordes no existieran.

In [ ]:
# Parámetros físicos y espaciales
a = 1           # Difusividad térmica
L = 100         # Longitud del dominio
N = 1000        # Número de puntos
dx = L / N
x = np.linspace(-L/2, L/2 - dx, N)

# Números de onda discretos
kappa = (2 * np.pi / L) * np.fft.fftshift(np.arange(-N//2, N//2))

# Condición inicial: pulso rectangular
u0 = np.zeros_like(x)
start = int((L/2 - L/10) / dx)
end = int((L/2 + L/10) / dx)
u0[start:end] = 1

# Transformada de Fourier de la condición inicial
u0_hat = np.fft.fft(u0)

# RHS de la ecuación del calor en el dominio de Fourier
def rhs_heat(t, uhat):
    return -a**2 * (kappa**2) * uhat

# Simulación temporal
t_span = (0, 10)
t_eval = np.arange(0, 10.1, 0.1)
sol = sp.integrate.solve_ivp(rhs_heat, t_span, u0_hat, t_eval=t_eval, method='RK45')

# Transformada inversa para volver al dominio espacial
u = np.array([np.fft.ifft(uhat).real for uhat in sol.y.T])


In [ ]:
# Visualización tipo imágenes
plt.figure()
plt.imshow(np.flipud(u), aspect='auto', extent=[x[0], x[-1], t_eval[0], t_eval[-1]], cmap='jet')
plt.xlabel('Espacio')
plt.ylabel('Tiempo')
plt.title('Mapa de calor de la evolución')
plt.colorbar(label='Temperatura')
plt.show()

In [ ]:
# Tiempos discretos a graficar (cada 2 segundos, por ejemplo)
indices_tiempo = [i for i, t in enumerate(t_eval) if t % 2 == 0]

plt.figure(figsize=(5, 5))
for i in indices_tiempo:
    plt.plot(x, u[i], label=f't = {t_eval[i]:.1f}')

plt.xlabel('Espacio')
plt.ylabel('Temperatura')
plt.title('Perfiles de temperatura en tiempos discretos')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Visualización tipo waterfall
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
X, Y = np.meshgrid(x, t_eval[::10])  # X: espacio, Y: tiempo
Z = u[::10, :]                       # Z: temperatura
ax.plot_surface(X, Y, Z, cmap='jet')
ax.set_xlabel('Espacio')            # eje X
ax.set_ylabel('Tiempo')             # eje Y
ax.set_zlabel('Temperatura')        # eje Z
# T, X = np.meshgrid(t_eval[::10], x)
# U = u[::10, :]
# ax.plot_surface(T, X, U.T, cmap='jet')
# ax.set_xlabel('Tiempo')
# ax.set_ylabel('Espacio')
# ax.set_zlabel('Temperatura')
# # ax.view_init(elev=30, azim=-60)  # elevación y ángulo azimutal
plt.title('Evolución de la temperatura (Waterfall)')
plt.show()


In [ ]:
from mpl_toolkits.mplot3d import Axes3D  # ya incluido si usás plot_surface

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(projection='3d')

# Tiempos discretos (por ejemplo, cada 2 segundos)
indices_tiempo = [i for i, t in enumerate(t_eval) if t % 2 == 0]

for i in indices_tiempo:
    t_val = t_eval[i]
    ax.plot(x, [t_val]*len(x), u[i], label=f't = {t_val:.1f}')

ax.set_xlabel('Espacio')     # eje X
ax.set_ylabel('Tiempo')      # eje Y
ax.set_zlabel('Temperatura') # eje Z
ax.view_init(elev=30, azim=-60)
plt.title('Curvas de temperatura en tiempos discretos')
plt.legend()
# plt.tight_layout()
plt.show()

---
## Separación de variables

La idea de esta parte es ver que, aunque el método de separación de variables es analítico, podemos partir de esa base para numéricamente imponer las condiciones de contorno, encontrar las soluciones, y graficar.

El método de separación de variables es una técnica analítica poderosa utilizada para resolver ecuaciones en derivadas parciales (PDEs). Este método se aplica comúnmente a problemas donde las condiciones de contorno y las ecuaciones permiten la separación de las variables independientes. Aquí se explica el método y se proporciona un ejemplo típico.

El método de separación de variables se basa en la suposición de que la solución de la PDE puede ser escrita como un producto de funciones, cada una de las cuales depende de una sola variable independiente. Supongamos que tenemos una PDE en dos variables $x$ y $t$:

$$
u(x, t)
$$

La suposición básica es que $u(x, t)$ se puede escribir como un producto de dos funciones:

$$
u(x, t) = X(x) T(t)
$$

donde $X(x)$ es una función de $x$ y $T(t)$ es una función de $t$.

### Pasos del Método

1. **Suposición de la forma de la solución:**
   $u(x, t) = X(x) T(t)$
2. **Sustitución en la PDE:**
   Sustituyendo $u(x, t) = X(x) T(t)$ en la PDE dada.
3. **Separación de variables:**
   Separar las variables $x$ y $t$ de tal manera que cada lado de la ecuación resultante dependa solo de una variable independiente.
4. **Igualación a una constante de separación:**
   Igualar cada lado de la ecuación a una constante, ya que cada lado debe ser igual a una constante para que la igualdad sea válida para todos los valores de $x$ y $t$.
5. **Resolución de ODEs:**
   Resolver las ecuaciones diferenciales ordinarias (ODEs) resultantes para $X(x)$ y $T(t)$.
6. **Aplicación de condiciones de contorno e iniciales:**
   Usar las condiciones de contorno e iniciales para determinar las constantes de integración y construir la solución general.

Consideremos la ecuación de calor unidimensional: $ u_t = \alpha u_{xx} $, con condiciones de frontera $u(0, t) = 0$ y $u(L, t) = 0$, y una condición inicial $u(x, 0) = f(x)$. Supongamos que $u(x, t) = X(x) T(t)$.

Sustituyendo en la ecuación de calor:

$$
X(x) T'(t) = \alpha X''(x) T(t)
$$

Dividiendo ambos lados por $\alpha X(x) T(t)$:

$$
\frac{T'(t)}{\alpha T(t)} = \frac{X''(x)}{X(x)}
$$

El lado izquierdo depende solo de $t$ y el derecho solo de $x$, por lo que ambos deben ser iguales a una constante, digamos $-\lambda$.

$$
\frac{T'(t)}{\alpha T(t)} = -\lambda \quad \text{y} \quad \frac{X''(x)}{X(x)} = -\lambda
$$

Esto nos da dos ODEs:

$$
T'(t) + \alpha \lambda T(t) = 0 \\
X''(x) + \lambda X(x) = 0 \\
$$

Resolvemos primero para $X(x)$:
$X''(x) + \lambda X(x) = 0$

Las soluciones dependen del signo de $\lambda$:
- Si $\lambda = 0$:
  $X(x) = Ax + B$
- Si $\lambda = \mu^2$ ($\mu > 0$):
  $X(x) = C \cos(\mu x) + D \sin(\mu x)$
- Si $\lambda = -\mu^2$ ($\mu > 0$):
  $X(x) = E e^{\mu x} + F e^{-\mu x}$

Para que $X(0) = 0$ y $X(L) = 0$:

$$
X(0) = 0 \Rightarrow C = 0 \\
X(L) = 0 \Rightarrow D \sin(\mu L) = 0 \Rightarrow \mu L = n\pi \Rightarrow \mu = \frac{n\pi}{L}
$$

Por lo tanto:

$$
X_n(x) = \sin\left(\frac{n\pi x}{L}\right)
$$

Luego resolvemos para $T(t)$:

$$T'(t) + \alpha \left(\frac{n\pi}{L}\right)^2 T(t) = 0 \\
T(t) = e^{-\alpha \left(\frac{n\pi}{L}\right)^2 t}
$$

La solución general es una suma de las soluciones de cada $n$:

$$
u(x, t) = \sum_{n=1}^{\infty} B_n \sin\left(\frac{n\pi x}{L}\right) e^{-\alpha \left(\frac{n\pi}{L}\right)^2 t}
$$

Las constantes $B_n$ se determinan a partir de la condición inicial $u(x, 0) = f(x)$:

$$
f(x) = \sum_{n=1}^{\infty} B_n \sin\left(\frac{n\pi x}{L}\right)
$$

Esto corresponde a la serie de Fourier de $f(x)$:

$$
B_n = \frac{2}{L} \int_0^L f(x) \sin\left(\frac{n\pi x}{L}\right) dx
$$

**Ejemplo**

Planteamos un ejemplo para parámetros y condiciones de contorno e iniciales arbitrarias.

In [ ]:
# Parámetros
L = 1.0
alpha = 0.01
N = 50  # Número de términos de la serie
t_final = 0.1
x = np.linspace(0, L, 100)
# Condición inicial
def f(x):
    return np.sin(np.pi * x / L)
# Cálculo de los coeficientes B_n
def B_n(n):
    return 2 / L * np.trapezoid(f(x) * np.sin(n * np.pi * x / L), x)
# Solución en t = t_final
u = np.zeros_like(x)
for n in range(1, N + 1):
    u += B_n(n) * np.sin(n * np.pi * x / L) * np.exp(-alpha * (n * np.pi / L)**2 * t_final)
# Visualización de la solución
plt.plot(x, u, label=f't={t_final}')
plt.xlabel('Posición x')
plt.ylabel('u(x, t)')
plt.title('Solución de la Ecuación de Calor')
plt.legend()
plt.show()

---
# Ecuación de Ondas

Consideremos ahora la ecuación de onda unidimensional:

$$
u_{tt} = c^2 u_{xx}
$$

Representa fenómenos con propagación con velocidad finita, como son las ondas mecánicas, electromagnéticas, acústicas. La información viaja a lo largo de curvas características (trayectorias de propagación). No suavizan perturbaciones: las ondas transportan la forma del pulso. Requiere dos condiciones iniciales (posición y velocidad, o equivalentes), y condiciones de contorno que deben ser compatibles con la dirección de propagación. Un ejemplo podría verse al mover una cuerda, y observar que las perturbaciones viajan con velocidad.

Modelan fenómenos de propagación como ondas acústicas, electromagnéticas o mecánicas. Tienen un carácter "oscilatorio" y mantienen formas bien definidas durante la propagación.


---
## CTCS

Discretizando en el espacio y el tiempo, obtenemos:

$$
\frac{u_i^{n+1} - 2u_i^n + u_i^{n-1}}{\Delta t^2} = c^2 \frac{u_{i+1}^n - 2u_i^n + u_{i-1}^n}{\Delta x^2}
$$

Despejando $u_i^{n+1}$:

$$
u_i^{n+1} = 2u_i^n - u_i^{n-1} + \left( \frac{c \Delta t}{\Delta x} \right)^2 \left( u_{i+1}^n - 2u_i^n + u_{i-1}^n \right)
$$

**Ejemplo**

En este código, se resuelve la ecuación de onda en un dominio unidimensional con una condición inicial de pulso central. La solución numérica se obtiene utilizando el esquema CTCS del método de diferencias finitas.

In [ ]:
# Parámetros de la malla
L = 1.0  # Longitud del dominio
T = 0.1  # Tiempo total de simulación
Nx = 50  # Número de puntos en el espacio
Nt = 100  # Número de puntos en el tiempo
c = 1.0  # Velocidad de la onda
dx = L / (Nx - 1)
dt = T / Nt
r = (c * dt / dx)**2
# Inicialización de la malla
x = np.linspace(0, L, Nx)
u = np.zeros(Nx)
u_new = np.zeros(Nx)
u_old = np.zeros(Nx)
# Condiciones iniciales
u[int(Nx/4):int(3*Nx/4)] = 1.0  # Pulso inicial en el centro
u_old[:] = u[:]
# Iteración temporal
for n in range(1, Nt):
    for i in range(1, Nx-1):
        u_new[i] = 2*u[i] - u_old[i] + r * (u[i+1] - 2*u[i] + u[i-1])
    u_old[:] = u[:]
    u[:] = u_new[:]
# Visualización del resultado
plt.plot(x, u, label=f'Tiempo t={T}')
plt.xlabel('Posición x')
plt.ylabel('u(x, t)')
plt.title('Solución de la ecuación de onda')
plt.legend()
plt.show()

---
### Ejercicio 3

Resuelva la ecuación de ondas unidimensional (1D) para una cuerda de longitud $L=1$ con velocidad de propagación $c=1$. El dominio espacial es $x \in [0, 1]$ y el tiempo $t \in [0, T]$, donde $T=2$ (dos periodos completos si es una onda estacionaria fundamental).

$$\frac{\partial^2 u}{\partial t^2} = c^2 \frac{\partial^2 u}{\partial x^2}$$

Considere condiciones de contorno de Dirichlet), ya que los extremos de la cuerda están fijos (ondas estacionarias): $u(0, t) = 0, \quad u(1, t) = 0$.
Como condición inicial, considere un pulso triangular, es decir, que la cuerda se desplaza inicialmente con una forma triangular y se suelta desde el reposo.
$$u(x, 0) = \begin{cases} 2h x/a & \text{si } 0 \le x \le a/2 \\ 2h (a-x)/(a-a/2) & \text{si } a/2 \le x \le a \end{cases}$$
Donde la perturbación inicial ocurre en $x \in [0, a]$, con $a=0.5$ y altura máxima $h=0.01$. Para $x > a$, $u(x, 0) = 0$.
La velocidad inicial es cero:
$$\frac{\partial u}{\partial t}(x, 0) = 0$$

3.1. Utilice $N_x = 100$ puntos espaciales. Calcule $\Delta x$.

3.2. Determine el paso de tiempo $\Delta t$ para que se cumpla la **condición de estabilidad de CFL** con un ratio $\lambda = c \Delta t / \Delta x = 0.9$.

3.3. Implemente el esquema de diferencias finitas explícito. Simule la evolución hasta $T=2$ y visualice la propagación de la onda (por ejemplo, con una animación).

In [ ]:
# # # COMPLETAR

---
### Ejercicio 4

Sea la misma ecuación de ondas 1D y dominio espacial $L=1$, $c=1$, $T=2$. Considere ahora condiciones de contorno de Neumann, es decir, que los extremos de la cuerda están libres (la derivada es cero), lo que permite que una onda viaje y se refleje como si el medio continuara (extremos "abiertos"):
$$\frac{\partial u}{\partial x}(0, t) = 0, \quad \frac{\partial u}{\partial x}(1, t) = 0$$

Si usamos como condición inicial un pulso gaussiano, la cuerda se desplaza inicialmente con un pulso gaussiano y se suelta desde el reposo.
$$u(x, 0) = A \exp\left(-\frac{(x-x_0)^2}{2\sigma^2}\right)$$
Donde $A=0.01$ es la amplitud, $x_0 = 0.25$ es la posición inicial del centro del pulso, y $\sigma=0.05$ es el ancho del pulso.
La velocidad inicial es cero:
$$\frac{\partial u}{\partial t}(x, 0) = 0$$

4.1. Usando los mismos parámetros $N_x=100$ y $\lambda=0.9$, implemente el mismo esquema de diferencias finitas explícito pero para la discretización de las condiciones de contorno de Neumann, y simule la evolución.

**Ayuda:**

Para la condición de Neumann $\partial u / \partial x = 0$ en el contorno se discretiza utilizando un **punto ficticio** (o **punto fantasma**).

En el contorno izquierdo ($x=0$):
$$\left.\frac{\partial u}{\partial x}\right|_{i=0} = 0 \quad \implies \quad \frac{u_1^n - u_{-1}^n}{2 \Delta x} = 0 \quad \implies \quad u_{-1}^n = u_1^n$$
El valor del punto ficticio $u_{-1}^n$ es igual al del primer punto interior $u_1^n$.

Sustituyendo $u_{-1}^n$ en la ecuación de evolución general (para $i=0$):
$$u_0^{n+1} = 2u_0^n - u_0^{n-1} + \lambda^2 (u_1^n - 2u_0^n + u_{-1}^n)$$
$$u_0^{n+1} = 2u_0^n - u_0^{n-1} + 2\lambda^2 (u_1^n - u_0^n)$$

En el contorno derecho ($x=L$):
$$\left.\frac{\partial u}{\partial x}\right|_{i=N_x} = 0 \quad \implies \quad \frac{u_{N_x+1}^n - u_{N_x-1}^n}{2 \Delta x} = 0 \quad \implies \quad u_{N_x+1}^n = u_{N_x-1}^n$$
Sustituyendo $u_{N_x+1}^n$ en la ecuación de evolución general (para $i=N_x$):
$$u_{N_x}^{n+1} = 2u_{N_x}^n - u_{N_x}^{n-1} + 2\lambda^2 (u_{N_x-1}^n - u_{N_x}^n)$$

La condición para el primer paso temporal ($n=1$) se adapta de manera similar, sustituyendo el punto ficticio en la fórmula especial.

4.2. Observe cómo el pulso viaja hacia el borde, se refleja y regresa como una onda viajera. Haga gráficos y animación.

In [ ]:
# # # COMPLETAR

---
## Métodos espectrales para la ecuación de ondas unidimensional

Sea la ecuación diferencial parcial lineal simple
$$ u_{t} + cu_{x} = 0 $$
Cualquier condición inicial $u(0,x)$ se propagará hacia la derecha en el tiempo con velocidad $c$, ya que $u(t,x) = u(0,x − ct)$ es una solución.

Resolvemos la ecuación para una condición inicial dada por un pulso gaussiano.

In [ ]:
c = 2  # Wave speed
L = 20  # Length of domain
N = 1000  # Number of discretization points
dx = L / N
x = np.arange(-L/2, L/2, dx)  # Define x domain
# Define discrete wavenumbers
kappa = (2 * np.pi / L) * np.arange(-N/2, N/2)

Es posible integrar esta ecuación en el dominio de la transformada de Fourier.

In [ ]:
# Parámetros
c = 1.0   # velocidad de onda (ajusta según tu caso)
x = np.linspace(-10, 10, 512)   # dominio espacial
kappa = np.fft.fftfreq(len(x), d=(x[1]-x[0])) * 2*np.pi
kappa = np.fft.fftshift(kappa)   # reordenar frecuencias

# Condición inicial
u0 = 1/np.cosh(x)   # sech(x)
uhat0 = np.fft.fft(u0)

# Definición de RHS en dominio de Fourier
def rhsWave(t, uhat, kappa, c):
    return -c * 1j * kappa * uhat

# Definición de RHS en dominio espacial
def rhsWaveSpatial(t, u, kappa, c):
    uhat = np.fft.fft(u)
    duhat = 1j * kappa * uhat
    du = np.fft.ifft(duhat)
    return -c * du.real   # tomar parte real para evitar residuos numéricos

# Tiempo de simulación
dt = 0.025
t_span = (0, 100*dt)
t_eval = np.arange(0, 100*dt, dt)

# Simulación en dominio de Fourier
sol_fourier = sp.integrate.solve_ivp(rhsWave, t_span, uhat0, t_eval=t_eval, args=(kappa, c))

# Simulación en dominio espacial
sol_spatial = sp.integrate.solve_ivp(rhsWaveSpatial, t_span, u0, t_eval=t_eval, args=(kappa, c))

# Resultados
uhat = sol_fourier.y
u = sol_spatial.y

Sin embargo, también es posible integrar esta ecuación en el dominio espacial, simplemente utilizando la FFT para calcular las derivadas y luego transformar de vuelta. La solución $u(t,x)$ se representa gráficamente

In [ ]:
# Visualización tipo imágenes
plt.figure()
plt.imshow(np.flipud(u), aspect='auto', extent=[x[0], x[-1], t_eval[0], t_eval[-1]], cmap='jet')
plt.xlabel('Espacio')
plt.ylabel('Tiempo')
plt.title('Evolución de la onda')
plt.colorbar()
plt.show()

In [ ]:
# Tiempos discretos a graficar (cada 2 segundos, por ejemplo)
indices_tiempo = [i for i, t in enumerate(t_eval) if t % 0.5 == 0]

plt.figure(figsize=(5, 5))
for i in indices_tiempo:
    plt.plot(x, u[:, i], label=f't = {t_eval[i]:.1f}')

plt.xlabel('Espacio')
plt.ylabel('Amplitud')
plt.title('Curvas de onda en tiempos discretos')
plt.legend()
plt.show()

In [ ]:
# Visualización tipo waterfall
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(projection='3d')
X, Y = np.meshgrid(x, t_eval[::10])  # X: espacio, Y: tiempo
Z = u.T[::10, :]
ax.plot_surface(X, Y, Z, cmap='jet')
ax.set_xlabel('Espacio')
ax.set_ylabel('Tiempo')
ax.set_zlabel('Amplitud')
ax.set_title('Evolución de la onda en el tiempo')
plt.subplots_adjust(left=-0.1, right=0.9)
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D  # ya incluido si usás plot_surface

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(projection='3d')

# Tiempos discretos (por ejemplo, cada 2 segundos)
indices_tiempo = [i for i, t in enumerate(t_eval) if t % 0.5 == 0]

for i in indices_tiempo:
    t_val = t_eval[i]
    ax.plot(x, [t_val]*len(x), u[:, i], label=f't = {t_val:.1f}')

ax.set_xlabel('Espacio')     # eje X
ax.set_ylabel('Tiempo')      # eje Y
ax.set_zlabel('Amplitud')    # eje Z
ax.set_title('Evolución de la onda en el tiempo')
plt.legend()
plt.show()

---
# Ecuación de Laplace y de Poisson

Consideremos la siguiente ecuación, que cuando $f=0$ es la ecuación de Laplace, y si $f\neq0$ es la ecuación de Poisson:

$$
\nabla^2 u = f(x, y)
$$

Describen estados de equilibrio: electrostática, temperatura estacionaria, elasticidad estática. La información viene desde todos los puntos del dominio al mismo tiempo. Representan estados estacionarios sin dependencia temporal, como el potencial electrostático, temperatura en equilibrio, elasticidad sin movimiento. No hay propagación ni tiempo. La información se distribuye en todo el dominio a la vez. Las soluciones tienden a ser suaves y “promediadas”. Requiere fijar todo el borde (Dirichlet, Neumann o mixtas). Sin condiciones de borde adecuadas, la solución no es única. Como ejemplo se puede pensar en cómo estirar una membrana: si fijás el borde, el interior queda completamente determinado.

---
## Métodos iterativos

En las ecuaciones elípticas como Laplace y Poisson no hay evolución temporal, de modo que lo que obtenemos al discretizar es directamente un sistema lineal que debe satisfacer la consistencia en toda la malla. Para resolverlo con diferencias finitas podríamos usar métodos directos, pero pueden ser costosos en 2D o 3D. Por esto, es común y suele ser más conveniente usar métodos iterativos, que ajustan la solución punto a punto usando solo información local.

En este contexto, Gauss-Seidel es uno de los métodos más simples y efectivos. Cada vez que actualiza un nodo, usa inmediatamente ese valor para seguir corrigiendo los demás, lo que acelera la convergencia y refleja bien la idea física de las ecuaciones elípticas, donde la solución es un equilibrio suave determinado por lo que ocurre en los bordes. Por eso se vuelve una herramienta natural y práctica para problemas de Laplace y Poisson discretizados.

---
### Ejercicio 5

Resuelva la ecuación de Laplace bidimensional (2D) en un dominio cuadrado unitario $D = [0, 1] \times [0, 1]$.
$$\nabla^2 u = \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = 0$$

Considere condiciones de contorno (mixtas):
* **Dirichlet (tres lados):**
    * $u(x, 0) = 0$ (Base)
    * $u(0, y) = 0$ (Lado izquierdo)
    * $u(1, y) = 100 \sin(\pi y)$ (Lado derecho)
* **Neumann (un lado):**
    * $\frac{\partial u}{\partial y}(x, 1) = 0$ (Tapa superior, aislada)

Para resolver este problema, el Laplaciano se aproxima en el punto $(i, j)$ como:

$$\nabla^2 u \approx \frac{u_{i+1, j} - 2u_{i, j} + u_{i-1, j}}{\Delta x^2} + \frac{u_{i, j+1} - 2u_{i, j} + u_{i, j-1}}{\Delta y^2} = 0$$

Para $\Delta x = \Delta y = h$, la discretización de la ecuación de Laplace es:

$$u_{i, j}^{k+1} = \frac{1}{4} (u_{i+1, j}^k + u_{i-1, j}^{k+1} + u_{i, j+1}^k + u_{i, j-1}^{k+1})$$

donde:
* $k$ es el número de iteración.
* En Gauss-Seidel, se usan los valores más recientes ($k+1$) tan pronto como están disponibles.

Para la tapa superior ($y=1$, $j=N_y$), tenemos una condición de Neumann. Usamos entonces un punto ficticio $u_{i, N_y+1}$ por encima del dominio.

$$\left.\frac{\partial u}{\partial y}\right|_{i, j=N_y} = 0 \quad \implies \quad \frac{u_{i, N_y+1} - u_{i, N_y-1}}{2 \Delta y} = 0 \quad \implies \quad u_{i, N_y+1} = u_{i, N_y-1}$$

Sustituyendo el punto ficticio en la ecuación de evolución para el contorno superior ($j=N_y$):

$$u_{i, N_y}^{k+1} = \frac{1}{4} (u_{i+1, N_y}^k + u_{i-1, N_y}^{k+1} + u_{i, N_y-1}^k + u_{i, N_y-1}^{k+1})$$

En esta iteración se utilizan los valores de $u_{i, N_y-1}$ (punto interior adyacente) en lugar de $u_{i, N_y+1}$ (punto ficticio).

5.1. Discretice el dominio con $N_x = N_y = 50$ puntos interiores. Implemente el esquema de diferencias finitas de cinco puntos (5-point stencil) para el Laplaciano. Utilice el método iterativo Gauss-Seidel para encontrar la solución estacionaria $u(x, y)$. Establezca un criterio de convergencia (por ejemplo, cuando el error relativo $\le 10^{-6}$).

5.2. Visualice la solución $u(x, y)$ (por ejemplo, con un mapa de colores o un gráfico 3D).

In [ ]:
# # # COMPLETAR

---
## Métodos espectrales para la ecuación de Laplace/Poisson bidimensional

En las ecuaciones elípticas discretizadas con diferencias finitas, el operador de Laplace queda representado por una matriz grande y esparsa, por lo que resulta natural usar métodos iterativos como Gauss-Seidel o SOR: son simples, aprovechan la estructura local del stencil y evitan el costo alto de factorizar toda la matriz.

En cambio, en métodos espectrales la situación es distinta. La discretización transforma el problema en el espacio de los modos, donde el operador Laplaciano se vuelve diagonal (o casi diagonal). Esto significa que cada modo se puede resolver de manera directa, sin iteraciones, simplemente dividiendo por el valor del modo del operador. No tiene sentido aplicar Gauss-Seidel porque no existe la misma estructura de vecindad local, y porque la solución modal ya desacopla el problema. En espectrales, los métodos iterativos solo reaparecen cuando el problema tiene geometrías complicadas, condiciones mixtas, o cuando el sistema deja de ser diagonalizable de forma sencilla.


**Ejemplo**

Sea la ecuación de Poisson bidimensional (2D) en un dominio cuadrado $D = [0, \pi] \times [0, \pi]$.
$$\nabla^2 u = f(x, y)$$

Supongamos condiciones de contorno de Dirichlet $u(x, 0) = u(x, \pi) = u(0, y) = u(\pi, y) = 0$, y una *fuente* ($f(x, y)$) tal que la solución analítica es $u(x, y) = \sin(2x) \sin(3y)$:

$$f(x, y) = \nabla^2 (\sin(2x) \sin(3y)) = \left(\frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2}\right) (\sin(2x) \sin(3y))$$
$$f(x, y) = (-4 \sin(2x) \sin(3y)) + (-9 \sin(2x) \sin(3y))$$
$$f(x, y) = -13 \sin(2x) \sin(3y)$$

En el espacio espectral, la ecuación de Poisson se convierte en un conjunto de ecuaciones algebraicas no acopladas para los coeficientes de Fourier $\tilde{u}_{k, l}$:

$$(-k^2 - l^2) \tilde{u}_{k, l} = \tilde{f}_{k, l}$$

donde $k$ y $l$ son los números de onda espaciales.

Implementemos un método espectral basado en la Transformada Discreta de Fourier (DFT) para funciones impares o Transformada Discreta de Seno (DST), que satisface las condiciones de Dirichlet de forma natural. Discreticemos el dominio con $N_x = N_y = 64$ puntos. Transformemos la función fuente $f(x, y)$ al espacio espectral. Resolvamos la ecuación en el espacio espectral para obtener los coeficientes espectrales de $u(x, y)$. Apliquemos la Transformada Inversa para obtener la solución numérica $u_{num}(x, y)$. Por último, calculemos el **error relativo** (por ejemplo, norma $L_2$) entre la solución numérica y la solución analítica $u_{analítica}(x, y) = \sin(2x) \sin(3y)$.

In [ ]:
L = np.pi # Dominio [0, pi] x [0, pi]
N = 64 # Número de puntos interiores
h = L / (N + 1) # Paso espacial
x = np.linspace(h, L - h, N) # Puntos interiores
X, Y = np.meshgrid(x, x)

# Función Fuente f(x, y) = -13 * sin(2x) * sin(3y)
f_xy = -13.0 * np.sin(2 * X) * np.sin(3 * Y)

# Se aplica DST-I, que asume condiciones de contorno de Dirichlet (cero)
f_tilde = sp.fft.dstn(f_xy, type=1)

# Solución en el Espacio Espectral
k = np.arange(1, N + 1) # Números de onda (1, 2, ..., N)
K, L_wavenum = np.meshgrid(k, k)

# Denominador de la solución espectral: - (k^2 + l^2)
# Nota: La DST mapea los índices (i, j) a los números de onda (k, l)
Denominator = -(K**2 + L_wavenum**2)

# Solución espectral: u_tilde = f_tilde / Denominator
u_tilde = f_tilde / Denominator

# Transformada Inversa (IDST)
# u_num se obtiene aplicando la IDST
u_num = sp.fft.idstn(u_tilde, type=1) / (4 * (N + 1)**2)
# Nota: La normalización por el factor de (4 * (N+1)^2) depende de la convención de la DST,
# la IDST en scipy ya incluye un factor de N/2, se ajusta según el tipo de DST.

# Cálculo del Error (Verificación)
# Solución analítica: u_anal(x, y) = sin(2x) * sin(3y)
u_anal = np.sin(2 * X) * np.sin(3 * Y)
# Error L2 (Norma)
error_l2 = np.sqrt(np.sum((u_num - u_anal)**2)) / np.sqrt(np.sum(u_anal**2))
# print(f"Error L2 (Spectral): {error_l2:.2e}")
# Se espera que este error sea muy pequeño (del orden de la precisión de la máquina)
# si el problema analítico y la base espectral coinciden.

In [ ]:
# X, Y: coordenadas (NxN)
# u_num: solución numérica (NxN)
# u_anal: solución analítica (NxN)
# error_l2: error L2 global

# Cálculo de la Magnitud del Error Local
error_abs = np.abs(u_num - u_anal)
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
# plt.suptitle(f"Ecuación de Poisson (Método Espectral) | Error L2: {error_l2:.2e}", fontsize=14)
ax1 = axes[0]
im1 = ax1.pcolormesh(X, Y, u_num, shading='auto', cmap='viridis')
ax1.set_xlabel("Posición $x$")
ax1.set_ylabel("Posición $y$")
# ax1.set_title("A. Solución Numérica $u_{num}(x, y)$")
# fig.colorbar(im1, ax=ax1, label="$u$")
ax2 = axes[1]
# La escala de color debe ser muy pequeña ya que el error espectral es minúsculo
im2 = ax2.pcolormesh(X, Y, error_abs, shading='auto', cmap='cividis')
ax2.set_xlabel("Posición $x$")
ax2.set_ylabel("Posición $y$")
# ax2.set_title("B. Error Absoluto Local")
# fig.colorbar(im2, ax=ax2, label="|Error|", format="%.1e")
# plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.subplots_adjust(wspace=0.3)
plt.show()